## When regridding the SIF dataset linearly from 720 to 1440 lons (upsampling), large values appear near masked areas. Why?

### Current workaround is in place and works roughly as expected: any values outside of the input range will be masked.

Area weighted regridding works correctly.

In [ ]:
import logging
import os
import sys
import warnings
from collections import namedtuple
from itertools import combinations

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from alepython import ale_plot
from joblib import Memory
from loguru import logger as loguru_logger
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from tqdm import tqdm

import wildfires.analysis
from wildfires.analysis import *
from wildfires.dask_cx1 import get_client
from wildfires.data import *
from wildfires.logging_config import enable_logging
from wildfires.qstat import get_ncpus
from wildfires.utils import *

loguru_logger.enable("alepython")
loguru_logger.remove()
loguru_logger.add(sys.stderr, level="WARNING")

logger = logging.getLogger(__name__)

enable_logging("jupyter")

warnings.filterwarnings("ignore", ".*Collapsing a non-contiguous coordinate.*")
warnings.filterwarnings("ignore", ".*DEFAULT_SPHERICAL_EARTH_RADIUS*")
warnings.filterwarnings("ignore", ".*guessing contiguous bounds*")

normal_coast_linewidth = 0.5
mpl.rc("figure", figsize=(14, 6))
mpl.rc("font", size=9.0)

In [ ]:
sif = GlobFluo_SIF()

In [ ]:
linear_sif = regrid(sif.cube, area_weighted=False, verbose=True)

In [ ]:
_ = cube_plotting(sif.cube.data.data[4])

In [ ]:
_ = cube_plotting(sif.cube[4])

In [ ]:
unmasked_diff = sif.cube.data.data[4] - sif.cube.data.data[3]
min_diff = np.min(unmasked_diff)
max_diff = np.max(unmasked_diff)
_ = cube_plotting(
    unmasked_diff,
    title=f"Unmasked Diff Index 4 - Index 3, Min, Max: {(min_diff, max_diff)}",
)

In [ ]:
_ = cube_plotting(linear_sif.data[4], title="SIF - Linear Regridding")

In [ ]:
_ = cube_plotting(sif.cube.data.mask, title="Raw SIF mask")

In [ ]:
_ = cube_plotting(np.all(sif.cube.data.mask, axis=0), title="ALL Raw SIF mask")

In [ ]:
_ = cube_plotting(
    np.all(linear_sif.data.mask, axis=0), title="ALL Linear Regridded SIF mask"
)

### Test Nearest-Neighbour Regridding

In [ ]:
nn_sif = regrid(sif.cube, verbose=True)

In [ ]:
_ = cube_plotting(nn_sif.data[4], title="SIF - Nearest-Neighbour Regridding")

In [ ]:
_ = cube_plotting(nn_sif, title="SIF - Nearest-Neighbour Regridding")

In [ ]:
print(
    "Max Original:",
    sif.cube.collapsed(("time", "latitude", "longitude"), iris.analysis.MAX).data,
)
print(
    "Max NN:",
    nn_sif.collapsed(("time", "latitude", "longitude"), iris.analysis.MAX).data,
)
print(
    "Max Linear:",
    linear_sif.collapsed(("time", "latitude", "longitude"), iris.analysis.MAX).data,
)

print(
    "Min Original:",
    sif.cube.collapsed(("time", "latitude", "longitude"), iris.analysis.MIN).data,
)
print(
    "Min NN:",
    nn_sif.collapsed(("time", "latitude", "longitude"), iris.analysis.MIN).data,
)
print(
    "Min Linear:",
    linear_sif.collapsed(("time", "latitude", "longitude"), iris.analysis.MIN).data,
)